In [1]:
import numpy as np
import pandas as pd

# Byte-Pair Encoding

In [2]:
corpus = {'low_' : 5, 'lowest_' : 2, 'newer_' : 6, 'wider_' : 3, 'new_' : 2}

In [ ]:
vocab = []
for key in corpus.keys():
    for char in key:
        vocab.append(char)
vocab = set(vocab)

{'_', 'd', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w'}

In [ ]:
def bpe(corpus, k):
    